In [37]:
import os
import sys
import csv
import operator
import time
import datetime
import platform
import threading

In [38]:
def writeText(text, path, mode = 'w'):
    with open (path, mode, encoding = 'utf-8') as textout:
        textout.write((text))
        
def writeJson(json, path, mode = 'w'):
    with open(path, mode) as file:
        file.write(json.dumps(json))
        
def writeCsv(listOut, outputFile):
    with open (outputFile, "w", newline='', encoding = 'utf-8') as outputfile:
        writer = csv.writer(outputfile, delimiter = ",")
        for element in listOut:
            writer.writerow(element)
            
def getTxt(path):
    return open(path, 'r').read()

def getCsv(path, delim = ','):
    list_return = []
    with open (path, encoding = 'utf-8') as file:
        csvreader = csv.reader(file, delimiter = delim)        
        for i, line in enumerate(csvreader):
            list_return.append(line)
    return list_return

In [39]:
def getFloatCsv(path, delim = ','):
    list_return = []
    with open (path, encoding = 'utf-8') as file:
        csvreader = csv.reader(file, delimiter = delim)        
        for i, line in enumerate(csvreader):
            list_return.append([float(x) for x in line])
    return list_return

### data import

In [40]:
if platform.system() == 'Windows':
    feat = getFloatCsv('..\\output\\feat.csv')
else:
    feat = getFloatCsv('../output/feat.csv')

In [41]:
if platform.system() == 'Windows':
    label = getCsv('..\\output\\labels.csv')
else:
    label = getCsv('../output/labels.csv')

In [42]:
# ll = []
# for line in label:
#     for word in line:
#         if word not in ll:
#             ll.append(word)
# len(ll)'

### numpy prep

In [43]:
import numpy as np

In [44]:
X = np.array(feat)

### sklearn prep

In [45]:
from sklearn.preprocessing import MultiLabelBinarizer

In [46]:
from sklearn import metrics

In [47]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(label)
y = multilabel_binarizer.transform(label)

In [48]:
X, y

(array([[-0.40620422,  0.37869263,  0.58084106, ...,  0.03137207,
          0.64916992, -1.32333374],
        [-0.50666809,  0.45892334,  0.73953247, ..., -0.08215332,
          1.0402832 , -1.47055054],
        [-0.44416809,  0.43939209,  0.61599731, ..., -0.01086426,
          0.69067383, -1.39877319],
        ...,
        [ 2.33901978,  7.3309021 , -1.46138   , ..., -2.83006287,
          7.28421021,  1.01919556],
        [-1.52600098,  1.57382202,  2.0201416 , ..., -1.18414307,
          1.93450928, -0.99139404],
        [-1.51477051,  1.66415405,  1.97241211, ..., -1.46881104,
          2.1239624 , -1.10540771]]), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]))

In [49]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [50]:
print('Dims training set: ', train_X.shape, train_y.shape)
print('Dims training set: ', test_X.shape, test_y.shape)

Dims training set:  (2694, 300) (2694, 125)
Dims training set:  (674, 300) (674, 125)


### RandomForest

In [51]:
from sklearn.ensemble import RandomForestClassifier

# Grid Search

In [52]:
def gridResultRFC(data, n_est1, n_est2, n_est_grid, depth1, depth2, depth_grid, min_leaf1, min_leaf2, min_leaf_grid, filename = 'log_rf_grid', thread = 1):
    train_X, test_X, train_y, test_y = data[0], data[1], data[2], data[3]
    acc_best = 0
    prec_best = 0
    acc_params_best = [0,0,0]
    prec_params_best = [0,0,0]
    str_log = 'Started at:\n' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S') + '\nThread: ' + str(thread) + '\n'
    print(str_log)
    est_runs = (n_est2-n_est1)/n_est_grid
    depth_runs = (depth2-depth1)/depth_grid
    leaf_runs = (min_leaf2-min_leaf1)/min_leaf_grid
    str_tmp = 'Grid Search will test ' + str(est_runs*depth_runs*leaf_runs) + ' combinations.\n'
    str_log += str_tmp
    print(str_tmp)
    
    est_act = n_est1
    depth_act = depth1
    leaf_act = min_leaf1
    
    while (est_act < n_est2):
        depth_act = depth1
        while (depth_act < depth2):
            leaf_act = min_leaf1
            while (leaf_act < min_leaf2):
                rf = RandomForestClassifier(n_estimators=est_act, max_depth=depth_act, min_samples_leaf=leaf_act)
                rf.fit(train_X, train_y)
                pred_rf = rf.predict(test_X)        
                prec =  metrics.precision_score(test_y, pred_rf, average="samples")
                acc = metrics.accuracy_score(test_y, pred_rf)
                print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
                str_tmp = '----------------------\n'+ 'Thread: ' + str(thread) + '\n' + 'Params: '+str(est_act)+','+str(depth_act)+','+str(leaf_act)+'\n'+'Accuracy: '+str(acc)+';'+' Precision: '+str(prec)+'\n'
                print('Accuracy: ', str(acc_best), '\nParams: ', str(acc_params_best))
                print('Precision: ', str(prec_best), '\nParams: ', str(prec_params_best), '\n')
                str_log += str_tmp
                print(str_tmp)
                if acc_best <= acc:
                    acc_best = acc
                    acc_params_best = [est_act,depth_act,leaf_act]
                if prec_best <= prec:
                    prec_best = prec
                    prec_params_best = [est_act,depth_act,leaf_act]
                leaf_act += min_leaf_grid
            depth_act += depth_grid
        est_act += n_est_grid
        
    str_tmp = '==========================\n==========================\n\n\n'+'Accuracy: '+str(acc_best)+'\nParams: '+str(acc_params_best) + '\n' + 'Precision: '+str(prec_best)+'\nParams: '+str(prec_params_best)+'\n'+'Ended at:\n'+datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
    str_log += str_tmp
    
    print('==========================\n==========================\n\n')
    print('Thread: ', thread)
    print('Accuracy: ', str(acc_best), '\nParams: ', str(acc_params_best))
    print('Precision: ', str(prec_best), '\nParams: ', str(prec_params_best), '\n\n')
    print('Ended at:\n' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    
    writeText(str_log, ('..\\output\\' + filename + '.txt'))             
    

In [53]:
test_train_set = [train_X, test_X, train_y, test_y]

In [54]:
thread_list = []

t1 = threading.Thread(target=gridResultRFC, args=(test_train_set, 100, 151, 25, 40, 45, 2, 1, 3, 3, 'log_rf_grid_1.2', 1))
t2 = threading.Thread(target=gridResultRFC, args=(test_train_set, 100, 151, 25, 46, 51, 2, 1, 3, 3, 'log_rf_grid_2.2', 2))
t3 = threading.Thread(target=gridResultRFC, args=(test_train_set, 100, 151, 25, 52, 57, 2, 1, 3, 3, 'log_rf_grid_3.2', 3))
t4 = threading.Thread(target=gridResultRFC, args=(test_train_set, 175, 226, 25, 40, 45, 2, 1, 3, 3, 'log_rf_grid_4.2', 4))
t5 = threading.Thread(target=gridResultRFC, args=(test_train_set, 175, 226, 25, 46, 51, 2, 1, 3, 3, 'log_rf_grid_5.2', 5))
t6 = threading.Thread(target=gridResultRFC, args=(test_train_set, 175, 226, 25, 52, 57, 2, 1, 3, 3, 'log_rf_grid_6.2', 6))


# Sticks the thread in a list so that it remains accessible
thread_list.append(t1)
thread_list.append(t2)
thread_list.append(t3)
thread_list.append(t4)
thread_list.append(t5)
thread_list.append(t6)

# for thread in thread_list:
#     thread.start()

# for thread in thread_list:
#     thread.join()

print('done')

done


In [55]:
# rf = RandomForestClassifier(n_estimators=125, max_depth=58, min_samples_leaf=1)

In [56]:
# rf.fit(train_X, train_y)
# pred_rf = rf.predict(test_X)

In [57]:
# prec =  metrics.precision_score(test_y, pred_rf, average="samples")
# acc = metrics.accuracy_score(test_y, pred_rf)
# print(acc,prec)

# kNN

In [58]:
from sklearn.neighbors import KNeighborsClassifier

In [59]:
n = 1
n_max =50
l_acc, l_prec = [], []
acc_n_best, prec_n_best, acc_best, prec_best = 0, 0, 0, 0
while n < 50:
    print('Evaluate n=', n)
    knn = KNeighborsClassifier(n_neighbors=n)
    knn.fit(train_X, train_y)
    
    pred_knn = knn.predict(test_X)
    prec =  metrics.precision_score(test_y, pred_knn, average="samples")
    acc = metrics.accuracy_score(test_y, pred_knn)
    
    l_acc.append(acc)
    l_prec.append(prec)
    
    if acc_best <= acc:
        acc_best = acc
        acc_n_best = n
    if prec_best <= prec:
        prec_best = prec
        prec_n_best = n
        
    n += 1

Evaluate n= 1
Evaluate n= 2


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


Evaluate n= 3
Evaluate n= 4
Evaluate n= 5
Evaluate n= 6
Evaluate n= 7
Evaluate n= 8
Evaluate n= 9
Evaluate n= 10
Evaluate n= 11
Evaluate n= 12
Evaluate n= 13
Evaluate n= 14
Evaluate n= 15
Evaluate n= 16
Evaluate n= 17
Evaluate n= 18
Evaluate n= 19
Evaluate n= 20
Evaluate n= 21
Evaluate n= 22
Evaluate n= 23
Evaluate n= 24
Evaluate n= 25
Evaluate n= 26
Evaluate n= 27
Evaluate n= 28
Evaluate n= 29
Evaluate n= 30
Evaluate n= 31
Evaluate n= 32
Evaluate n= 33
Evaluate n= 34
Evaluate n= 35
Evaluate n= 36
Evaluate n= 37
Evaluate n= 38
Evaluate n= 39
Evaluate n= 40
Evaluate n= 41
Evaluate n= 42
Evaluate n= 43
Evaluate n= 44
Evaluate n= 45
Evaluate n= 46
Evaluate n= 47
Evaluate n= 48
Evaluate n= 49


In [60]:
l_prec, l_acc

([0.8922264184875461,
  0.8635578442329184,
  0.838499556822999,
  0.831596914460416,
  0.8059846022754034,
  0.7893879982529832,
  0.7736821907074133,
  0.7716103382275489,
  0.7581936362351792,
  0.7358077382558094,
  0.7418943568795202,
  0.7240230837411846,
  0.7200277467339782,
  0.7077344020964199,
  0.7077873906508921,
  0.6950242141636801,
  0.689781879841227,
  0.6850517682120056,
  0.6758776831477128,
  0.6698581833596671,
  0.6645169370688658,
  0.6654318727760864,
  0.6641460171875603,
  0.6562083317276195,
  0.6555424422264184,
  0.6464919971225609,
  0.6410023828792375,
  0.6341280010790397,
  0.6348698408416509,
  0.633400291597621,
  0.6371518812542551,
  0.6345536757999666,
  0.6373232109137154,
  0.635846596529089,
  0.6356487725923928,
  0.6289510193071024,
  0.6345642735108611,
  0.6287673256515987,
  0.6292618854933394,
  0.6151669300037252,
  0.6173606561588758,
  0.6102884504219815,
  0.6117474019551171,
  0.607501252456742,
  0.6057950210027362,
  0.603403470910

In [61]:
acc_n_best, acc_best

(1, 0.6943620178041543)

In [62]:
prec_n_best, prec_best

(1, 0.8922264184875461)

In [63]:
# pred_knn = knn.predict(test_X)
# print('Precision: ', metrics.precision_score(test_y, pred_knn, average="samples"))

In [64]:
# print('Accuracy: ', metrics.accuracy_score(test_y, pred_knn))

# OnevsRest

In [65]:
# from sklearn import metrics
# from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.svm import SVC
# from numpy import array

# clf = OneVsRestClassifier(SVC(probability=True, gamma='auto'))
# clf.fit(train_X, train_y)
# predictions = clf.predict(test_X)

# my_metrics = metrics.classification_report(test_y, predictions)

# # print(my_metrics)

In [66]:
# print(clf.score(test_X, test_y, sample_weight=None))

# Neural Network

In [67]:
from sklearn.neural_network import MLPClassifier

In [68]:
# mclf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)

In [69]:
# mclf.fit(train_X, train_y)
# predictionsm = mclf.predict(test_X)

In [70]:
# print('Precision: ', metrics.precision_score(test_y, predictionsm,average='samples'))

In [71]:
# print('Accuracy: ', metrics.accuracy_score(test_y, predictionsm))

In [72]:
def gridResultNN(data, hl1, hl2, hlg, filename, thread):
    train_X, test_X, train_y, test_y = data[0], data[1], data[2], data[3]
    hla = hl1
    l_acc, l_prec = [], []
    str_tmp = ''
    acc_n_best, prec_n_best, acc_best, prec_best = 0, 0, 0, 0
    str_log = 'Started at:\n' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S') + '\nThread: ' + str(thread) + '\n'
    print(str_log)
    while hla < hl2:
        print('Evaluate Hidden Layers = ', hla)
        mclf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(hla,), random_state=1)
        mclf.fit(train_X, train_y)
        predictionsm = mclf.predict(test_X)
        prec =  metrics.precision_score(test_y, predictionsm, average="samples")
        acc = metrics.accuracy_score(test_y, predictionsm)

        l_acc.append(acc)
        l_prec.append(prec)
    
        if acc_best <= acc:
            acc_best = acc
            acc_n_best = hla
        if prec_best <= prec:
            prec_best = prec
            prec_n_best = hla
        
        str_tmp += 'Evaluate Hidden Layers = ' + str(hla) + '\n' + 'Acc: ' + str(acc) + ', Prec: ' + str(prec) + '\n'
        print(str_tmp)
        str_log += str_tmp
        hla += hlg
        
    print('==========================\n==========================\n\n')
    print('Thread: ', thread)
    print('Accuracy: ', str(acc_best), '\nParams: ', str(acc_n_best))
    print('Precision: ', str(prec_best), '\nParams: ', str(prec_n_best), '\n\n')
        
    str_tmp = ('Ended at:\n' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    print(str_tmp)
    str_log += '\n' + str_tmp
    
    writeText(str_log, ('..\\output\\' + filename + '.txt')) 

In [73]:
thread_list = []

t1 = threading.Thread(target=gridResultNN, args=(test_train_set, 2, 51, 2, 'log_nn_grid_1', 1))
t2 = threading.Thread(target=gridResultNN, args=(test_train_set, 52, 200, 10, 'log_nn_grid_2', 2))
t3 = threading.Thread(target=gridResultNN, args=(test_train_set, 220, 500, 30, 'log_nn_grid_3', 3))
t4 = threading.Thread(target=gridResultNN, args=(test_train_set, 520, 1000, 50, 'log_nn_grid_4', 4))
t5 = threading.Thread(target=gridResultNN, args=(test_train_set, 1000, 1800, 75, 'log_nn_grid_5', 5))
t6 = threading.Thread(target=gridResultNN, args=(test_train_set, 1800, 3000, 180, 'log_nn_grid_6', 6))


# Sticks the thread in a list so that it remains accessible
thread_list.append(t1)
thread_list.append(t2)
thread_list.append(t3)
thread_list.append(t4)
thread_list.append(t5)
thread_list.append(t6)

# Starts threads
for thread in thread_list:
    thread.start()

# This blocks the calling thread until the thread whose join() method is called is terminated.
# From http://docs.python.org/2/library/threading.html#thread-objects
for thread in thread_list:
    thread.join()

# Demonstrates that the main process waited for threads to complete
print('done')

Started at:
2018-11-13 00:46:23
Thread: 1
Started at:
2018-11-13 00:46:23
Thread: 2

Evaluate Hidden Layers =  2
Started at:
2018-11-13 00:46:23
Thread: 3

Evaluate Hidden Layers =  52

Evaluate Hidden Layers =  220
Started at:
2018-11-13 00:46:23
Thread: 4
Started at:
2018-11-13 00:46:23
Thread: 5

Evaluate Hidden Layers =  520
Started at:
2018-11-13 00:46:23
Thread: 6

Evaluate Hidden Layers =  1000

Evaluate Hidden Layers =  1800


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


Evaluate Hidden Layers = 2
Acc: 0.0, Prec: 0.0

Evaluate Hidden Layers =  4
Evaluate Hidden Layers = 2
Acc: 0.0, Prec: 0.0
Evaluate Hidden Layers = 4
Acc: 0.0, Prec: 0.0

Evaluate Hidden Layers =  6
Evaluate Hidden Layers = 52
Acc: 0.5089020771513353, Prec: 0.8395069281196879

Evaluate Hidden Layers =  62
Evaluate Hidden Layers = 220
Acc: 0.6513353115727003, Prec: 0.893339178131463

Evaluate Hidden Layers =  250
Evaluate Hidden Layers = 520
Acc: 0.6394658753709199, Prec: 0.8846025430224242

Evaluate Hidden Layers =  570
Evaluate Hidden Layers = 2
Acc: 0.0, Prec: 0.0
Evaluate Hidden Layers = 4
Acc: 0.0, Prec: 0.0
Evaluate Hidden Layers = 6
Acc: 0.08605341246290801, Prec: 0.3409750857451154

Evaluate Hidden Layers =  8
Evaluate Hidden Layers = 1000
Acc: 0.6216617210682492, Prec: 0.8861711774000132

Evaluate Hidden Layers =  1075
Evaluate Hidden Layers = 52
Acc: 0.5089020771513353, Prec: 0.8395069281196879
Evaluate Hidden Layers = 62
Acc: 0.5801186943620178, Prec: 0.8684981437948798

Eval

Evaluate Hidden Layers = 2
Acc: 0.0, Prec: 0.0
Evaluate Hidden Layers = 4
Acc: 0.0, Prec: 0.0
Evaluate Hidden Layers = 6
Acc: 0.08605341246290801, Prec: 0.3409750857451154
Evaluate Hidden Layers = 8
Acc: 0.050445103857566766, Prec: 0.22458398789782977
Evaluate Hidden Layers = 10
Acc: 0.04599406528189911, Prec: 0.19185270128382734
Evaluate Hidden Layers = 12
Acc: 0.06528189910979229, Prec: 0.15231779756139702
Evaluate Hidden Layers = 14
Acc: 0.07863501483679525, Prec: 0.38862062764139915
Evaluate Hidden Layers = 16
Acc: 0.19881305637982197, Prec: 0.6162832222179403
Evaluate Hidden Layers = 18
Acc: 0.22551928783382788, Prec: 0.6576758655950051

Evaluate Hidden Layers =  20
Evaluate Hidden Layers = 1000
Acc: 0.6216617210682492, Prec: 0.8861711774000132
Evaluate Hidden Layers = 1075
Acc: 0.6394658753709199, Prec: 0.8908169229385846
Evaluate Hidden Layers = 1150
Acc: 0.6201780415430267, Prec: 0.887087411846314
Evaluate Hidden Layers = 1225
Acc: 0.6231454005934718, Prec: 0.8861645278559225



Evaluate Hidden Layers = 52
Acc: 0.5089020771513353, Prec: 0.8395069281196879
Evaluate Hidden Layers = 62
Acc: 0.5801186943620178, Prec: 0.8684981437948798
Evaluate Hidden Layers = 72
Acc: 0.6023738872403561, Prec: 0.8768491399797037
Evaluate Hidden Layers = 82
Acc: 0.5949554896142433, Prec: 0.8744819022834385
Evaluate Hidden Layers = 92
Acc: 0.6261127596439169, Prec: 0.8894674810846918
Evaluate Hidden Layers = 102
Acc: 0.615727002967359, Prec: 0.8833744766608268
Evaluate Hidden Layers = 112
Acc: 0.6305637982195845, Prec: 0.8845660397960101
Evaluate Hidden Layers = 122
Acc: 0.6142433234421365, Prec: 0.885486274358678
Evaluate Hidden Layers = 132
Acc: 0.6305637982195845, Prec: 0.8902234511284957

Evaluate Hidden Layers =  142
Evaluate Hidden Layers = 2
Acc: 0.0, Prec: 0.0
Evaluate Hidden Layers = 4
Acc: 0.0, Prec: 0.0
Evaluate Hidden Layers = 6
Acc: 0.08605341246290801, Prec: 0.3409750857451154
Evaluate Hidden Layers = 8
Acc: 0.050445103857566766, Prec: 0.22458398789782977
Evaluate Hidd

Evaluate Hidden Layers = 2
Acc: 0.0, Prec: 0.0
Evaluate Hidden Layers = 4
Acc: 0.0, Prec: 0.0
Evaluate Hidden Layers = 6
Acc: 0.08605341246290801, Prec: 0.3409750857451154
Evaluate Hidden Layers = 8
Acc: 0.050445103857566766, Prec: 0.22458398789782977
Evaluate Hidden Layers = 10
Acc: 0.04599406528189911, Prec: 0.19185270128382734
Evaluate Hidden Layers = 12
Acc: 0.06528189910979229, Prec: 0.15231779756139702
Evaluate Hidden Layers = 14
Acc: 0.07863501483679525, Prec: 0.38862062764139915
Evaluate Hidden Layers = 16
Acc: 0.19881305637982197, Prec: 0.6162832222179403
Evaluate Hidden Layers = 18
Acc: 0.22551928783382788, Prec: 0.6576758655950051
Evaluate Hidden Layers = 20
Acc: 0.21068249258160238, Prec: 0.6551185337906406
Evaluate Hidden Layers = 22
Acc: 0.4169139465875371, Prec: 0.7914527855922515
Evaluate Hidden Layers = 24
Acc: 0.33086053412462907, Prec: 0.717617990318287
Evaluate Hidden Layers = 26
Acc: 0.4421364985163205, Prec: 0.8012798073999855
Evaluate Hidden Layers = 28
Acc: 0.41

Evaluate Hidden Layers = 52
Acc: 0.5089020771513353, Prec: 0.8395069281196879
Evaluate Hidden Layers = 62
Acc: 0.5801186943620178, Prec: 0.8684981437948798
Evaluate Hidden Layers = 72
Acc: 0.6023738872403561, Prec: 0.8768491399797037
Evaluate Hidden Layers = 82
Acc: 0.5949554896142433, Prec: 0.8744819022834385
Evaluate Hidden Layers = 92
Acc: 0.6261127596439169, Prec: 0.8894674810846918
Evaluate Hidden Layers = 102
Acc: 0.615727002967359, Prec: 0.8833744766608268
Evaluate Hidden Layers = 112
Acc: 0.6305637982195845, Prec: 0.8845660397960101
Evaluate Hidden Layers = 122
Acc: 0.6142433234421365, Prec: 0.885486274358678
Evaluate Hidden Layers = 132
Acc: 0.6305637982195845, Prec: 0.8902234511284957
Evaluate Hidden Layers = 142
Acc: 0.6364985163204748, Prec: 0.8875351651316042
Evaluate Hidden Layers = 152
Acc: 0.6172106824925816, Prec: 0.8837623800531811
Evaluate Hidden Layers = 162
Acc: 0.6394658753709199, Prec: 0.8913591724793505
Evaluate Hidden Layers = 172
Acc: 0.6379821958456974, Prec:

Evaluate Hidden Layers = 2
Acc: 0.0, Prec: 0.0
Evaluate Hidden Layers = 4
Acc: 0.0, Prec: 0.0
Evaluate Hidden Layers = 6
Acc: 0.08605341246290801, Prec: 0.3409750857451154
Evaluate Hidden Layers = 8
Acc: 0.050445103857566766, Prec: 0.22458398789782977
Evaluate Hidden Layers = 10
Acc: 0.04599406528189911, Prec: 0.19185270128382734
Evaluate Hidden Layers = 12
Acc: 0.06528189910979229, Prec: 0.15231779756139702
Evaluate Hidden Layers = 14
Acc: 0.07863501483679525, Prec: 0.38862062764139915
Evaluate Hidden Layers = 16
Acc: 0.19881305637982197, Prec: 0.6162832222179403
Evaluate Hidden Layers = 18
Acc: 0.22551928783382788, Prec: 0.6576758655950051
Evaluate Hidden Layers = 20
Acc: 0.21068249258160238, Prec: 0.6551185337906406
Evaluate Hidden Layers = 22
Acc: 0.4169139465875371, Prec: 0.7914527855922515
Evaluate Hidden Layers = 24
Acc: 0.33086053412462907, Prec: 0.717617990318287
Evaluate Hidden Layers = 26
Acc: 0.4421364985163205, Prec: 0.8012798073999855
Evaluate Hidden Layers = 28
Acc: 0.41

Evaluate Hidden Layers = 1000
Acc: 0.6216617210682492, Prec: 0.8861711774000132
Evaluate Hidden Layers = 1075
Acc: 0.6394658753709199, Prec: 0.8908169229385846
Evaluate Hidden Layers = 1150
Acc: 0.6201780415430267, Prec: 0.887087411846314
Evaluate Hidden Layers = 1225
Acc: 0.6231454005934718, Prec: 0.8861645278559225
Evaluate Hidden Layers = 1300
Acc: 0.6439169139465876, Prec: 0.8944033649337803
Evaluate Hidden Layers = 1375
Acc: 0.6083086053412463, Prec: 0.8843169935899905
Evaluate Hidden Layers = 1450
Acc: 0.6364985163204748, Prec: 0.8898101404036122
Evaluate Hidden Layers = 1525
Acc: 0.6231454005934718, Prec: 0.892222885917248
Evaluate Hidden Layers = 1600
Acc: 0.6068249258160238, Prec: 0.8846534709108894
Evaluate Hidden Layers = 1675
Acc: 0.6350148367952523, Prec: 0.8905979035800994
Evaluate Hidden Layers = 1750
Acc: 0.612759643916914, Prec: 0.8838877862987656



Thread:  5
Accuracy:  0.6439169139465876 
Params:  1300
Precision:  0.8944033649337803 
Params:  1300 


Ended at:
2018-